In [33]:
import numpy as np
from scipy.io import loadmat
import os
import scipy
from sklearn.discriminant_analysis import _cov
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
import discr_inverted as di

In [34]:

file = 'C:\\Users\\danne\\Documents\\UofT\\Eric\\s01_cr.mat'
annots = loadmat(file)
data_cr = annots['mat_cr']
lbls_cr = annots['lbls_cr']
file = 'C:\\Users\\danne\\Documents\\UofT\\Eric\\s01_hit.mat'
annots = loadmat(file)
data_hit = annots['mat_hit']
lbls_hit = annots['lbls_hit']

In [8]:
data_cr[:,:,1].dtype

dtype('<f8')

In [11]:
data_cr[:10,:10,0]

array([[ 2.36459216,  2.05525583,  1.98892347,  3.3126951 ,  2.06129508,
         4.18472622,  5.4665759 ,  4.08502673,  2.60481374,  1.78339754],
       [-0.58605786, -6.40673121,  0.96764969,  1.32164307, -0.38687716,
         1.63959729,  1.64271531, -0.13023909,  0.15840538,  0.37057366],
       [-3.85238956, -9.18742719, -1.23501245, -5.07052672, -2.37274125,
        -0.95538032, -1.79766196,  1.43017471, -0.15996197, -0.81145067],
       [-0.72055083,  0.13436532, -0.65881815, -0.3172713 , -1.32634309,
        -1.54644487, -1.07712082, -0.26233789, -0.7091426 , -1.34215494],
       [-2.14299647, -2.13109849, -2.28747191, -1.44246804, -2.76526569,
        -4.29254368, -4.60984803, -4.85816482, -3.40633849, -2.18747075],
       [ 1.32161092, -1.54403533,  1.63702524,  2.42975724,  0.73551974,
         1.72117956,  1.25620364,  2.504259  ,  1.06587824,  0.80311262],
       [ 0.20965545,  0.31109262,  3.06678657, -1.74791409, -0.48839341,
        -0.692412  ,  0.04452148, -0.24298623

In [14]:
temp = _cov(data_cr[:,:,0], shrinkage='auto')
temp = scipy.linalg.fractional_matrix_power(temp, -0.5)
temp.dtype

dtype('float64')

In [39]:
svm = SVC(kernel='linear')
CV = di.ShuffleBinLeaveOneOut
out = list()

X = data_cr.copy()
y = lbls_cr.copy()-1

n_conditions = len(np.unique(y))
n_sensors = X.shape[1]
n_time = X.shape[2]
n_perm = 40
n_pseudo = 10
cv = CV(y, n_iter=n_perm, n_pseudo=n_pseudo)
result = np.full((n_perm, n_conditions, n_conditions), np.nan)
for f, (train_indices, test_indices) in enumerate(cv.split(X)):
    print(f)
    Xpseudo_train = np.full((len(train_indices), n_sensors, n_time), np.nan)
    Xpseudo_test = np.full((len(test_indices), n_sensors, n_time), np.nan)
    for i, ind in enumerate(train_indices):
        Xpseudo_train[i, :, :] = np.mean(X[ind, :, :], axis=0)
    for i, ind in enumerate(test_indices):
        Xpseudo_test[i, :, :] = np.mean(X[ind, :, :], axis=0)



    sigma_conditions = cv.labels_pseudo_train[0, :, n_pseudo-1:].flatten()
    sigma_ = np.empty((n_conditions, n_sensors, n_sensors))
    for k,c in enumerate(np.unique(y)):
                # compute sigma for each time point, then average across time
        a = [_cov(Xpseudo_train[sigma_conditions==c, :, t], shrinkage='auto') for t in range(n_time)]
        sigma_[k] = np.mean(a, axis=0)
    sigma = sigma_.mean(axis=0)  # average across conditions
    sigma_inv = scipy.linalg.fractional_matrix_power(sigma, -0.5)
    Xpseudo_train = (Xpseudo_train.swapaxes(1, 2) @ sigma_inv).swapaxes(1, 2)
    Xpseudo_test = (Xpseudo_test.swapaxes(1, 2) @ sigma_inv).swapaxes(1, 2)

    for c1 in range(n_conditions-1):
        for c2 in range(min(c1 + 1, n_conditions-1), n_conditions):
                        # 3. Fit the classifier using training data
            data_train = Xpseudo_train[cv.ind_pseudo_train[c1, c2], :, :]
            data_train = np.reshape(data_train, (data_train.shape[0], data_train.shape[1]*data_train.shape[2]), order='F')
            svm.fit(data_train, cv.labels_pseudo_train[c1, c2])                            

                        # 4. Compute and store classification accuracies
            data_test = Xpseudo_test[cv.ind_pseudo_test[c1, c2], :, :]
            data_test = np.reshape(data_test, (data_test.shape[0], data_test.shape[1]*data_test.shape[2]), order='F')
            result[f, c1, c2] = np.mean(svm.predict(data_test) == cv.labels_pseudo_test[c1, c2]) - 0.5                            


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39


In [25]:
a = np.array(a)

In [30]:
b = np.mean(a, axis=0)

In [31]:
b.dtype

dtype('float64')

In [40]:
result.shape

(40, 3, 3)

In [41]:
a = np.nanmean(result,axis=0)

C:\Users\danne\Anaconda3\envs\eeg_analysis\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: Mean of empty slice
  """Entry point for launching an IPython kernel.


In [42]:
a

array([[  nan, 0.025, 0.1  ],
       [  nan,   nan, 0.175],
       [  nan,   nan,   nan]])